# Elasto-plastic isotropic hardening model

Verification of the Legendre transform

In [ ]:
%matplotlib widget
from bmcs_matmod import GSM
import matplotlib.pylab as plt 
import sympy as sp
import numpy as np
from bmcs_matmod.gsm.potentials.potential1d_t_e_vp_lih import Potential1D_T_E_VP_LIH_SymbExpr 
sp.init_printing()

## Helmholtz free energy

In [ ]:
p1d = Potential1D_T_E_VP_LIH_SymbExpr()
p1d.F_

## Gibbs free energy

In [ ]:
eps_a = p1d.eps_a
sig_a = p1d.sig_a
dF_du = p1d.F_.diff(eps_a)
u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
                            zip(sig_a, eps_a, dF_du)])
subs_u_sig_ = dict(zip(eps_a, u_sig_))
sp.simplify(dF_du), subs_u_sig_

In [ ]:
sig_x_eps_ = (sig_a.T * eps_a)[0]
G_expr = p1d.F_ - sig_x_eps_
G_ = sp.simplify(G_expr.subs(subs_u_sig_))
G_

In [ ]:
G_sig = sp.simplify(p1d.F_.subs(subs_u_sig_).args[0] - sig_x_eps_.subs(subs_u_sig_))
G_sig

In [ ]:
sp.simplify((G_sig +  + p1d.sig * p1d.eps).subs(p1d.sig, p1d.E*(p1d.eps - p1d.eps_p)))

## GSM drivers

In [ ]:
gsm_F = GSM(
    name = 'gsm_F_1d_t_e_vp_lkh',
    u_vars = p1d.eps_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = p1d.Sig_signs,
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

gsm_G = GSM(
    name = 'gsm_G_1d_t_e_vp_lkh',
    u_vars = p1d.sig_a,
    sig_vars = p1d.eps_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_signs = p1d.Sig_signs,
    Sig_vars = p1d.Sig_vars,
    F_expr = G_,
    sig_sign = sp.Rational(-1),
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

In [ ]:
gsm_F.Sig_, gsm_G.Sig_

In [ ]:
(gsm_F.df_dSig_.T * gsm_F.dSig_dEps_ * gsm_F.Phi_, 
 gsm_G.df_dSig_.T * gsm_G.dSig_dEps_ * gsm_G.Phi_)

In [ ]:
gsm_F.df_dlambda_, gsm_G.df_dlambda_

In [ ]:
lambda_solved_F = gsm_F.f_ / gsm_F.df_dlambda_
lambda_solved_G = gsm_G.f_ / gsm_G.df_dlambda_

In [ ]:
subs_Sig_Eps_F = dict(zip(gsm_F.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
subs_Sig_Eps_G = dict(zip(gsm_G.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
subs_Sig_Eps_F, subs_Sig_Eps_G

In [ ]:
lambda_solved_F.subs(subs_Sig_Eps_F), lambda_solved_G.subs(subs_Sig_Eps_G)

In [ ]:
gsm_F.f_, gsm_G.f_

In [ ]:
gsm_F.df_dSig_,gsm_G.df_dSig_

In [ ]:
gsm_F.Phi_, gsm_G.Phi_

In [ ]:
gsm_G.df_dEps_.T * gsm_G.Phi_

In [ ]:
subs_Sig_Eps_F = dict(zip(gsm_F.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
delta_Eps_F = sp.simplify((gsm_F.f_/ gsm_F.df_dlambda_)* gsm_F.Phi_).subs(subs_Sig_Eps_F)
subs_Sig_Eps_G = dict(zip(gsm_G.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
delta_Eps_G = sp.simplify((gsm_G.f_/ gsm_G.df_dlambda_)* gsm_G.Phi_).subs(subs_Sig_Eps_G)
lambda_solved_F, lambda_solved_G

In [ ]:
gsm_F.Phi_, gsm_G.Phi_

In [ ]:
gsm_G.F_expr.diff(gsm_G.u_vars), gsm_G.sig_

In [ ]:
gsm_F.save_to_disk()
gsm_G.save_to_disk()

## Monotonic loading

In [ ]:
_f_c = 44
_f_t = -0.1 * _f_c
_X_0 = (_f_c + _f_t) / 2
_f_s = (_f_c - _f_t) / 2
_E = 50000
_K_factor = 1e-5
_K = _E * _K_factor
material_params = dict(
    E_=_E, 
    K_lin_= _K / 10, # _E / 5,
    f_c_=_f_s,
    eta_=20000,
    T_0_=20,
    C_v_= 1e+6, # 0.01, # 0.0001, 
    d_N_ = 1,
    beta_=0 # 0.5,
)
p1d.mparams

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t, lam_t, (d_t_t, d_u_ta) = response
    _u_atI, _Eps_btI, _Sig_btI, _dF_dEps_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb, _dF_dEps_t)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI, lam_t, (d_t_t, d_u_ta)

gsm_F.vp_on = True
gsm_F.update_at_k = False

def get_cyclic_load(max_s, max_t, n_cycles, n_incr):
    # Generating loading history
    s_arr = np.tile(np.array([-1, 1]), n_cycles) * np.linspace(0, max_s, 2 * n_cycles)
    s_arr = np.interp(np.linspace(0, max_t, n_incr * len(s_arr)), np.linspace(0, max_t, len(s_arr)), s_arr)

    # time array as input
    t_arr = np.linspace(0, max_t, len(s_arr))
    return s_arr, t_arr

In [ ]:

def generate_cyclic_load(max_s, min_s, freq, total_cycles, points_per_cycle):
    # Calculate the time for one cycle
    total_time = total_cycles / freq

    # Calculate the mean value and amplitude
    mean_value = (max_s + min_s) / 2
    amplitude = (max_s - min_s) / 2

    # Calculate the initial loading slope
    slope = 2 * np.pi * freq * amplitude
    
    # Time arrays for linear increase and sinusoidal part
    initial_duration = mean_value / slope
    initial_points = int(initial_duration * freq * points_per_cycle)
    total_points = int(total_time * freq * points_per_cycle)
    
    # Generate the initial linear increase
    initial_t = np.linspace(0, initial_duration, initial_points, endpoint=False)
    initial_loading = slope * initial_t

    # Generate the sinusoidal loading
    sinusoidal_t = np.linspace(0, total_time, total_points, endpoint=False)
    sinusoidal_loading = mean_value + amplitude * np.sin(2 * np.pi * freq * sinusoidal_t)

    # Combine the initial linear increase with the sinusoidal loading
    t_full = np.concatenate((initial_t, sinusoidal_t + initial_duration))
    s_full = np.concatenate((initial_loading, sinusoidal_loading))
    
    return s_full, t_full

In [ ]:
material_params

In [ ]:
max_s = 0.002
u_t_F, t_t_F = get_cyclic_load( n_cycles=1, max_s=max_s, 
                                max_t=1, n_incr = 3)
# u_t_F, t_t_F = generate_cyclic_load(max_s, -max_s, 1, 10, 30)
T_t_F = 20 + t_t_F * 0
response_values_F = gsm_run(gsm_F, u_t_F[:,np.newaxis], T_t_F, t_t_F, **material_params)

In [ ]:
_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F, lam_t_F, (d_t_t, d_u_ta) = response_values_F 
argmax_sig_ = np.argmax(_sig_atI_F)
t_t_G = _t_t_F # [:argmax_sig_+1]
u_ta_G = _sig_atI_F[...].reshape(-1, 1) # [0, :argmax_sig_+1, 0].reshape(-1, 1)
T_t_G = 20 + t_t_G * 0
response_values_G = gsm_run(gsm_G, u_ta_G, T_t_G, t_t_G, **material_params)

In [ ]:
fig, ((ax_eps, ax_sig), (ax_lam, ax_z), (ax_eps_p, ax_sig_eps)) = plt.subplots(3,2, figsize=(12,10), tight_layout=True)

_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F, _lam_t_F, (_d_t_F, _d_u_ta_F) = response_values_F 
_u_p_atI_F, _z_atI_F = gsm_F.Eps_as_blocks(_Eps_btI_F)
_, _Z_atI_F = gsm_F.Eps_as_blocks(_Sig_btI_F)

_t_t_G, _sig_atI_G, _u_atI_G, _T_t_G, _Eps_btI_G, _Sig_btI_G, _dF_dEps_btI_G, _lam_t_G, (_d_t_G, _d_u_ta_G) = response_values_G 
_u_p_atI_G, _z_atI_G = gsm_G.Eps_as_blocks(_Eps_btI_G)
_, _Z_atI_G = gsm_G.Eps_as_blocks(_Sig_btI_G)

ax_sig_eps.plot(_u_atI_F[0, :, 0], _sig_atI_F[0, :, 0], 'o-', label='F', color='blue',)
ax_sig_eps.plot(_u_atI_G[0, :, 0], _sig_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_sig_eps.legend()
ax_sig_eps.set_title(r'strain-strain')
ax_sig_eps.set_ylabel(r'$\sigma$')
ax_sig_eps.set_xlabel(r'$\varepsilon$')

ax_eps.plot(_t_t_F, _u_atI_F[0, :, 0], 'o-',label='F', color='blue',)
ax_eps.plot(_t_t_G, _u_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_eps.legend()
ax_eps.set_title(r'strain-time')
ax_eps.set_ylabel(r'$\varepsilon$')
ax_eps.set_xlabel(r'$t$')

ax_sig.plot(_t_t_F, _sig_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_sig.plot(_t_t_G, _sig_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_sig.legend()
ax_sig.set_title(r'stress-time')
ax_sig.set_ylabel(r'$\sigma$')
ax_sig.set_xlabel(r'$t$')

ax_lam.plot(_t_t_F, _lam_t_F, 'o-', label='F', color='blue')
ax_lam.plot(_t_t_G, _lam_t_G, 'o-', label='G', color='red')
ax_lam.legend()
ax_lam.set_title(r'$\lambda$-time')
ax_lam.set_ylabel(r'$\lambda$')
ax_lam.set_xlabel(r'$t$')

ax_z.plot(_t_t_F, _z_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_z.plot(_t_t_G, _z_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_z.legend()
ax_z.set_title(r'z-time')
ax_z.set_ylabel(r'$z$')
ax_z.set_xlabel(r'$t$')

ax_eps_p.plot(_t_t_F, _u_p_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_eps_p.plot(_t_t_G, _u_p_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_eps_p.legend()
ax_eps_p.set_title(r'$\varepsilon_\mathrm{p}$-time')
ax_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$')
ax_eps_p.set_xlabel(r'$t$')
